In [28]:
# Basic imports

import tqdm as notebook_tqdm

import numpy as np
import importlib
import functions
import pandas as pd

# Data engineering
from sklearn.preprocessing import StandardScaler

# Machine Learning
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb

# Neural Networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Functions imports
importlib.reload(functions)
from functions import *

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

In [29]:
x = pd.read_csv("../data/processed/x_processed.csv")  # 10605 rows × 10 columns
x_challenge = pd.read_csv("../data/processed/x_challenge_processed.csv")  # 4942 rows × 10 columns
x_challenge_unprocessed = pd.read_csv("../data/raw/X_test_GgyECq8.csv")
y = pd.read_csv("../data/raw/y_train_jJtXgMX.csv")  # 10605 rows × 2 columns

In [30]:
y.head(5)

,DELIVERY_START,spot_id_delta
0,2022-01-01 02:00:00+01:00,-36.874770
1,2022-01-01 03:00:00+01:00,-12.643588
2,2022-01-01 04:00:00+01:00,-1.950193
3,2022-01-01 05:00:00+01:00,1.938272
4,2022-01-01 06:00:00+01:00,0.199907


Préparation des données

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x.drop(columns=['DELIVERY_START']), y, test_size=.1)

In [32]:
model = Sequential()
model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Couche de sortie pour la régression

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

/Users/hugopuybareau/Documents/GitHub/ELMY_DATA_CHALLENGE/env3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
# Entraînement du modèle
model.fit(x_train, y_train['spot_id_delta'], epochs=100, batch_size=32, validation_split=0.2)

# Évaluation du modèle sur l'ensemble de test
mse = model.evaluate(x_test, y_test['spot_id_delta'])
print(f'Mean Squared Error on Test Set: {mse}')

Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step - loss: nan - val_loss: nan
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step - loss: nan - val_loss: nan
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step - loss: nan - val_loss: nan
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: nan - val_loss: nan
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step - loss: nan - val_loss: nan
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step - loss: nan - val_loss: nan
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step - loss: nan - val_loss: nan
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - loss: nan - val_loss: nan
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: nan - val_loss: nan
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - loss: nan - val_loss: nan
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step - loss: nan - val_loss: nan
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step - loss: nan - val_loss: nan
E

In [34]:
# Prédictions
y_challenge = pd.DataFrame({ # Faut prendre 'x_challenge_unprocessed' parce que sinon y'a le décalage horaire
    'DELIVERY_START' : x_challenge_unprocessed['DELIVERY_START']  
    })

y_challenge['spot_id_delta'] = model.predict(x_challenge.drop(columns=['DELIVERY_START']))


y_challenge.head(5)

155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step


,DELIVERY_START,spot_id_delta
0,2023-04-02 00:00:00+02:00,NaN
1,2023-04-02 01:00:00+02:00,NaN
2,2023-04-02 02:00:00+02:00,NaN
3,2023-04-02 03:00:00+02:00,NaN
4,2023-04-02 04:00:00+02:00,NaN


In [35]:
y_challenge.to_csv("../data/predicted/y_challenge_11.csv", index=False)